By Vasyl Storchak

In [170]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 250)
np.set_printoptions(suppress=True)
# pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.float_format', str)


In [171]:
def answer_one():
    # load file from colab location
    # energy = pd.read_excel('/content/Energy Indicators.xls',
    energy = pd.read_excel('Energy Indicators.xls',
                            skiprows=16,      # skip nondata
                            nrows=227,
                            header=[0, 1])    # multi header
    # drop unused column
    energy = energy.iloc[:, 2:]
    # rename columns
    energy_columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.columns=energy_columns
    # check data
    # print(energy.head())
    # set np.NaN for missing data
    for it in range(1, 4):
        energy.iloc[:, it] = energy.iloc[:, it].apply(lambda x: x if x != '...' else np.nan)
    # convert to gigajoules
    energy.iloc[:, 1] = energy.iloc[:, 1].apply(lambda x: x * 1_000_000)

    # check data
    # print(energy)
    # print(energy.iloc[-10])
    # print('#' * 20)

    # rename countrys
    dict_rename_country = {'Republic of Korea': 'South Korea',
        'United States of America': 'United States',
        'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
        'China, Hong Kong Special Administrative Region': 'Hong Kong'}
    # print(energy[energy[energy_columns[0]].str.startswith('United States of America')])
    for key, val in dict_rename_country.items():
        # print(energy[energy[energy_columns[0]].str.startswith(key)])
        energy[energy_columns[0]] = energy[energy_columns[0]].apply(lambda x: val if x.startswith(key) else x)
        # print(energy[energy[energy_columns[0]].str.startswith(val)])
        # print('$' * 20)

    # print(energy[energy[energy_columns[0]].str.contains('\(')])
    # print(energy[energy[energy_columns[0]].str.contains('1')])
    # clean up breakets & digits
    energy[energy_columns[0]] = energy[energy_columns[0]].replace([r'\(.*?\)', r'\d+'], '', regex=True).str.strip()
    # print(energy[energy[energy_columns[0]].str.contains('\(')])
    # print(energy[energy[energy_columns[0]].str.contains('1')])
    # print(energy.iloc[[24, 70, 98, 111, 133, 183, 197, 221]])

    GDP = pd.read_csv('world_bank.csv', skiprows=4)
    # drop unused column
    GDP = GDP.iloc[:, :-1]

    # print(GDP.head())
    GDP_col = ["Country Name","Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970",
               "1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990",
               "1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010",
               "2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"]
    # print(GDP[GDP[GDP_col[0]] == 'Korea, Rep.'])
    
    dict_rename_country = {'Korea, Rep.': 'South Korea',
        'Iran, Islamic Rep.': 'Iran',
        'Hong Kong SAR, China': 'Hong Kong'}

    for key, val in dict_rename_country.items():
        GDP[GDP_col[0]] = GDP[GDP_col[0]].apply(lambda x: val if x == key else x)
    
    # print(GDP[GDP[GDP_col[0]] == 'South Korea'])
    # print(GDP[GDP[GDP_col[0]] == 'Iran'])

    ScimEn = pd.read_excel('scimagojr country rank 1996-2023.xlsx')

    GDP_filter_col = ["Country Name","Country Code","Indicator Name","Indicator Code","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015"]

    # print(ScimEn.head())

    df_merged = pd.merge(ScimEn.iloc[:15], energy, on='Country', how='inner')
    # print(df_merged.head())
    df_merged = pd.merge(df_merged, GDP[GDP_filter_col], 
                         left_on='Country', right_on='Country Name', 
                         how='inner')
    # print(df_merged[['Country', 'Country Name']])
    # df_merged = df_merged.drop('Country Name', axis=1)
    # print(df_merged.head())
    # print(df_merged.info())

    result_col = ['Country', 'Rank', 'Documents', 'Citable documents', 'Citations', 
                  'Self-citations', 'Citations per document', 'H index', 
                  'Energy Supply', 'Energy Supply per Capita', '% Renewable', 
                  '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

    result = df_merged[result_col]
    # print(result.info())
    result = result.set_index(result.columns[0])
    # print(result.info())
    # print(result)
    # result.to_csv('myout.csv')
    return result

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,...,2011,2012,2013,2014,2015
Country,,,,,,,,,,,
China,1,402188,400158,5077232,3511673,...,8275592638197.0,8926363432826.3,9619598215288.48,10333926843443.8,11061572618578.7
United States,2,208042,203356,3540288,1004877,...,16594704135000.0,16974575729000.0,17334068403000.0,17771549056000.0,18295019000000.0
India,3,81853,79757,968523,369256,...,1616399198833.92,1704596203504.32,1813453530766.63,1947834564908.95,2103588360044.94
Japan,4,58342,57735,717946,154848,...,4219912322037.65,4277925608850.6,4363702302795.86,4376627829637.51,4444930651964.18
United Kingdom,5,56288,54713,1095371,179764,...,2685966732014.03,2726497926337.32,2775572748536.75,2864242231205.47,2927911140916.73
Germany,6,50906,49773,777362,160302,...,3268207907666.64,3283708733629.98,3296391223767.13,3367906396055.51,3423568450957.03
Russian Federation,7,46186,45868,217996,96688,...,1304443243520.47,1356935163515.02,1380755103919.45,1390921151162.46,1363482182197.71
Canada,8,41209,40390,915491,142691,...,1443745912933.41,1469093201563.91,1503261570659.63,1546457292386.53,1556508816217.14
Italy,9,38700,36909,639473,147302,...,1923230167189.95,1863124631879.35,1829252559153.33,1829227175717.41,1845428048839.1


In [172]:
def answer_two():
    answer_o = answer_one()
    avgGDP = 0
    GDP_col = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    answer_o["Mean GDP"] = answer_o[GDP_col].mean(axis=1, numeric_only = True)

    avgGDP = answer_o['Mean GDP'] #.astype(np.int64)
    avgGDP = avgGDP.sort_values(ascending=False)
    # print(avgGDP.info())
    
    return avgGDP

answer_two()

Country
United States          16785492905900.0
China                 7951523944937.497
Japan                4289022297963.5835
Germany               3227306871509.983
United Kingdom       2734808569333.2495
France                2350499782286.544
Italy                 1905325390490.941
Brazil                 1691014735367.17
India                 1590546137454.646
Canada                1441727035741.363
Russian Federation   1291318938811.9148
South Korea            1274793247284.92
Australia            1204458454001.0452
Spain                1193341716237.9302
Iran                 393919118962.19336
Name: Mean GDP, dtype: float64

In [173]:
def answer_tree():
    top_15 = answer_one()
    avgGDP = answer_two()
    df = pd.merge(avgGDP.iloc[5:6], top_15,
                  left_index=True, right_index=True,
                  how='inner')
    print(df['2015'])
    print(df['2006'])
    result = np.float64(df['2015'].iloc[0] - df['2006'].iloc[0])
    return result

answer_tree()

Country
France   2442483452642.5
Name: 2015, dtype: float64
Country
France   2267844863251.48
Name: 2006, dtype: float64


np.float64(174638589391.02002)

In [174]:
def answer_four():
    top_15 = answer_one()
    result_col = ['Citations', 'Self-citations']
    top_15['Ratio citations'] = top_15[result_col[1]] / top_15[result_col[0]]
    result = top_15['Ratio citations'].sort_values(ascending=False)
    
    return (result.index[0], result.iloc[0])

answer_four()

('China', np.float64(0.6916510807463594))

In [175]:
def answer_five():
    top_15 = answer_one()
    cols = ['Energy Supply', 'Energy Supply per Capita']
    # print(top_15[cols])
    top_15['Population'] = top_15[cols[0]] / top_15[cols[1]]
    result = top_15.sort_values('Population', ascending=False)
    return result.index[2]

answer_five()

'United States'

In [176]:
def answer_six():
    top_15 = answer_one()
    cols = ['Energy Supply', 'Energy Supply per Capita', 'Citable documents per person']
    top_15['Population'] = top_15[cols[0]] / top_15[cols[1]]
    top_15[cols[2]] = top_15['Citable documents'] / top_15['Population']
    result = top_15[cols[2]].corr(top_15[cols[1]])

    return result

answer_six()

np.float64(0.7019601723071187)

In [177]:
def answer_seven():
    top_15 = answer_one()
    cols = ['Energy Supply', 'Energy Supply per Capita']
    top_15['Population'] = top_15[cols[0]] / top_15[cols[1]]
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    # ['Asia', 'Australia', 'Europe', 'North America', 'South America'] and columns ['size', 'sum', 'mean', 'std']

    tmp = dict()
    for key, val in ContinentDict.items():
        if val not in tmp:
            tmp[val] = []
        tmp[val].append(top_15.loc[key]['Population'])

    df = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in tmp.items() ]))
    df = df.T.reset_index().set_index('index')
    
    df['size'] = df.apply(lambda row: row.notna().sum(), axis=1)
    df['sum'] = df.apply(lambda row: row.sum(), axis=1)
    df['mean'] = df.apply(lambda row: row.mean(), axis=1)
    df['std'] = df.apply(lambda row: row.std(), axis=1)
    
    df = df.drop(df.columns[0:6], axis=1)
    return df

answer_seven()

,size,sum,mean,std
index,,,,
Asia,5,2898666391.6106005,828190397.6030287,1008968400.9766871
North America,2,352855251.4802495,176427625.74012476,159782180.34254685
Europe,6,457929673.2163719,114482418.30409297,135000938.41847947
Australia,1,23316018.316017315,15544012.21067821,10991275.733910287
South America,1,205915255.23728815,137276836.82485875,97069381.51159002
